In [1]:
import cv2 
import matplotlib.pyplot as plot

from matplotlib import pyplot as plt
from matplotlib import image as image

import easygui
import numpy as np

import glob

In [42]:
#Face Folder
datasetSize = 'newx64\\'
faceDir = datasetSize + 'Faces\\'
notFaceDir = datasetSize + 'NotFaces\\'

In [43]:
#Create the HOG descriptor
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)

In [44]:
#Get the images
faces = glob.glob(faceDir + '*.jpg')
nonFaces = glob.glob(notFaceDir + '*.jpg')
images = faces + nonFaces

#Create labels for the images
faceLabels = [1] * len(faces)
nonFaceLabels = [0] * len(nonFaces)
labels = faceLabels + nonFaceLabels

#Convert the lists into numpy arrays
images = np.array(images)
labels = np.array(labels)

#Shuffle the data
rand = np.random.RandomState(22)
shuffle = rand.permutation(len(images))
images, labels = images[shuffle], labels[shuffle]

print(np.unique(labels))
print(len(images))
print(len(faceLabels))

[0 1]
33187
7522


In [45]:
#Calculate HOG features for every image
descriptors = []

for img in images:
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    descriptor = hog.compute(img)
    descriptors.append(descriptor)

In [46]:
descriptors = np.squeeze(descriptors)

In [47]:
#Split the data into training and testing sets
#90% training - 10% testing

trainingSize = int(0.8 * len(descriptors))

imagesTrain, imagesTest = np.split(images, [trainingSize])
labelsTrain, labelsTest = np.split(labels, [trainingSize])
descriptorsTrain, descriptorsTest = np.split(descriptors, [trainingSize])

# #Double check
# print(len(descriptorsTrain))
# print(len(descriptorsTest))
# print(len(descriptors))
# print(labelsTrain)

In [48]:
#Initialise the SVM classifier
SVM = cv2.ml.SVM_create()
SVM.setKernel(cv2.ml.SVM_RBF)
SVM.setGamma(0.50625)
SVM.setType(cv2.ml.SVM_C_SVC)
SVM.setC(2.0)

#training
SVM.train(descriptorsTrain, cv2.ml.ROW_SAMPLE, labelsTrain)

True

In [ ]:
#Get the accuracy of the predictions on the test set
output = SVM.predict(descriptorsTest)[1].ravel()

accuracy = (labelsTest == output).mean()
print('Percentage Accuracy: %.2f %%' % (accuracy*100))

In [ ]:
#Get a confusion matrix for the test set
#ACTUAL VALUES       -Face- -NotFace- 
#PREDICTED -Face-    --TP-- --FP-- 
#PREDICTED -NotFace- --FN-- --TN-- 

confusion = np.zeros((2, 2), np.int32)
for i, j in zip(labelsTest, output):
    confusion[int(i), int(j)] += 1
print('confusion matrix:')
print(confusion)


In [ ]:
#Get all the images that were incorrectly labelled 
#https://stackoverflow.com/questions/46615554/how-to-display-multiple-images-in-one-figure-correctly/46616645
#This does not display properly if there are too many unfortunately
incorrectedLabeledImages = []
for i, j, k in zip(labelsTest, output, imagesTest):
    if i != j:
        incorrectedLabeledImages.append(k)


#Then 
w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = len(incorrectedLabeledImages) 
rows = 1
for i in range(1, columns*rows +1):
    img = cv2.imread(incorrectedLabeledImages[i-1])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
#SAVE THE MODEL
SVM.save("model.xml")
SVMextr = SVM